# AutoEncoder template

## ======= Preparing the training and test sets =======

In [5]:
import numpy as np
import pandas as pd
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t', header = None).values
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t', header = None).values                          

#make data in the desired format rates(user, movie) = rating
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
def convert(data):
    new_data = []
    users = set(data[:, 0])
    for id_users in users:
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append([id_users]+list(ratings)) #keep track of users
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

#Converting the data into Torch tensors
import torch
training_set = torch.FloatTensor(training_set) #(a list of lists)
test_set = torch.FloatTensor(test_set)

## ====== Construct Stacked AutoEncoders =========

In [6]:
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5) #lr: learning_rate

## ======= Training SAE =========

In [7]:
nb_epoch = 30
nb_users = training_set.shape[0]
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user, 1:]).unsqueeze(0)
        target = input.clone()
        
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        train_loss += np.sqrt(loss.item()*mean_corrector)
        s += 1.
        loss.backward() #calculate derivatives 
        optimizer.step() #update parameters
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: 1.770640569109666
epoch: 2 loss: 1.0966356141431413
epoch: 3 loss: 1.053420498505396
epoch: 4 loss: 1.0383474670202826
epoch: 5 loss: 1.0306424877718154
epoch: 6 loss: 1.0264499865164698
epoch: 7 loss: 1.0239987508443718
epoch: 8 loss: 1.021853759876446
epoch: 9 loss: 1.0205251040679293
epoch: 10 loss: 1.0194876170039755
epoch: 11 loss: 1.0188551751699335
epoch: 12 loss: 1.018140193854717
epoch: 13 loss: 1.0178572825633225
epoch: 14 loss: 1.0175348825832486
epoch: 15 loss: 1.0170862452621856
epoch: 16 loss: 1.0171147498318571
epoch: 17 loss: 1.0165321298927124
epoch: 18 loss: 1.0164918875158349
epoch: 19 loss: 1.016348526353583
epoch: 20 loss: 1.0160784970747967
epoch: 21 loss: 1.0158412030953705
epoch: 22 loss: 1.01586272466297
epoch: 23 loss: 1.0158810453855036
epoch: 24 loss: 1.0159101670834985
epoch: 25 loss: 1.0158125484627185
epoch: 26 loss: 1.0155770538632702
epoch: 27 loss: 1.0152887547927167
epoch: 28 loss: 1.014919304209829
epoch: 29 loss: 1.0131459657515884
ep

## ========= Test SAE ========

In [8]:
test_loss = 0
s = 0.
ind_test = -1 #not 0!!!!
for id_users in test_set[:, 0]:
    id_user = int(id_users.item()) #extract the user id
    ind_test += 1  
    input = Variable(training_set[:, 1:][training_set[:, 0] == id_users]).unsqueeze(0)
    target = Variable(test_set[ind_test:ind_test+1, 1:]).unsqueeze(0)
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.item()*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s)+'\n')

test loss: 1.0185444770088083

